In [25]:
#𝐕𝐢𝐫𝐭𝐮𝐚𝐥 𝐄𝐱𝐡𝐢𝐛𝐢𝐭𝐢𝐨𝐧 𝐆𝐞𝐧𝐞𝐫𝐚𝐭𝐨𝐫

# This project aims to create a virtual exhibition generator that uses the Spotify API 
# and the Art Institute of Chicago's API. The goal is to allow users to select art genres
# and receive an accompanying music playlist to enhance their viewing experience.

In [26]:
#Importing necessary tools
# Spotipy: For accessing the Spotify API.
# Pandas: For data handling and manipulation.
# Urllib: For making HTTP requests to APIs.
# IPython.display: For displaying images and media in the Notebook.

import spotipy
import spotipy.util as util
import json
import webbrowser
import pandas as pd
import urllib.request
from IPython.display import display, Image

In [27]:
#Accessing Spotify API 

cred = "spotify_keys.json"
with open(cred,"r") as key_file:
    api_tokens = json.load(key_file)

key_file.close()

In [28]:
client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username'] 

In [29]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

In [30]:
sp = spotipy.Spotify(auth=token)

In [31]:
# Available genre seeds from Spotify are fetched, which will later be used to 
# generate music recommendations based on the selected art categories.

genre_seeds = sp.recommendation_genre_seeds()

In [32]:
genre_seeds

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [33]:
# Artwork data is loaded from a CSV file of data from the AIC Github,containing information 
# about their most popular art pieces. This will help form the foundation for my virtual exhibition."


art_data = pd.read_csv("AIC_data.csv")

In [34]:
display(art_data)

,id,title,main_reference_number,department_title,artist_title
0,869,The Watermill with the Great Red Roof,1894.1031,Painting and Sculpture of Europe,Meindert Hobbema
1,2189,Ready-to-Wear,1956.137,Arts of the Americas,Stuart Davis
2,2816,Interior at Nice,1956.339,Modern Art,Henri Matisse
3,4081,Gian Lodovico Madruzzo,1929.912,Painting and Sculpture of Europe,Giovanni Battista Moroni
4,4428,Mère Grégoire,1930.78,Painting and Sculpture of Europe,Gustave Courbet
...,...,...,...,...,...
417,234004,Untitled 72–12–A,2016.62,Contemporary Art,"Chung, Sang-Hwa"
418,234433,Involvement Series,2016.66,Contemporary Art,Wanda Pimentel
419,234781,Christ Carrying the Cross,2016.15,Painting and Sculpture of Europe,Sebastiano del Piombo
420,234972,Small Town by Day (Badische Kleinstadt bei Tage),2016.46,Modern Art,Georg Scholz


In [35]:
# Unique department titles are extracted from the art data, providing insight into how 
# the artworks are categorised.

departments = art_data["department_title"].unique()

print("Department Titles",departments)

Department Titles ['Painting and Sculpture of Europe' 'Arts of the Americas' 'Modern Art'
 'Contemporary Art' 'Arts of Asia' 'Photography and Media'
 'Prints and Drawings' 'Applied Arts of Europe' 'Textiles'
 'Arts of Africa' 'Architecture and Design' nan
 'Arts of the Ancient Mediterranean and Byzantium']


In [36]:
# A request is made to the Art Institute API to retrieve additional artwork details, 
# including image IDs and classifications, which enhances the dataset I am working with.

url = "https://api.artic.edu/api/v1/artworks?fields=id,title,artist_display,date_display,classification_titles,department_title,image_id"
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
all_art_data= json.loads(response.read())
all_art_data

{'pagination': {'total': 126319,
  'limit': 12,
  'offset': 0,
  'total_pages': 10527,
  'current_page': 1,
  'next_url': 'https://api.artic.edu/api/v1/artworks?page=2&fields=id%2Ctitle%2Cartist_display%2Cdate_display%2Cclassification_titles%2Cdepartment_title%2Cimage_id'},
 'data': [{'id': 184673,
   'title': '"Manhattan" Cocktail Set',
   'date_display': 'Designed 1934–35; produced c. 1939–41',
   'artist_display': 'Designed by Norman Bel Geddes (American, 1893–1958)\nManufactured by Revere Copper and Brass Company (American, founded 1801)\nRome, New York',
   'department_title': 'Arts of the Americas',
   'classification_titles': ['drinking vessel',
    'cocktail shaker',
    'tray',
    'american decorative arts',
    'decorative arts'],
   'image_id': '9f62b947-7f55-29e2-c617-5d64227db5f0'},
  {'id': 152747,
   'title': 'York Harbor, Coast of Maine',
   'date_display': '1877',
   'artist_display': 'Martin Johnson Heade (American, 1819–1904)',
   'department_title': 'Arts of the Am

In [37]:
# Analysing artword data

for artwork in all_art_data['data'][:20]: 
    
    print("Title:", artwork['title'])
    print("Artist:", artwork.get('artist_display',))
    print("Date:", artwork.get('date_display'))
    print("Classification:", artwork.get('classification_titles'))
    print("Department:", artwork.get('department_title'))
    print("\n")

Title: "Manhattan" Cocktail Set
Artist: Designed by Norman Bel Geddes (American, 1893–1958)
Manufactured by Revere Copper and Brass Company (American, founded 1801)
Rome, New York
Date: Designed 1934–35; produced c. 1939–41
Classification: ['drinking vessel', 'cocktail shaker', 'tray', 'american decorative arts', 'decorative arts']
Department: Arts of the Americas


Title: York Harbor, Coast of Maine
Artist: Martin Johnson Heade (American, 1819–1904)
Date: 1877
Classification: ['painting', 'american arts']
Department: Arts of the Americas


Title: Pair of Wine Coolers
Artist: Chased by Eugene J. Soligny (American, c. 1833–1901)
Tiffany and Company (American, founded 1837)
New York
Date: 1873
Classification: ['container', 'silver', 'american decorative arts', 'american arts', 'decorative arts']
Department: Arts of the Americas


Title: Timepiece
Artist: Works by Elnathan Taber (American, 1768–1854)
Roxbury, Massachusetts
Date: 1802–5
Classification: ['furniture accessory', 'american dec

In [38]:
#What are the unique department titles in this data set?
art_departments = [department_titles['department_title'] for department_titles in all_art_data['data']]
art_departments

['Arts of the Americas',
 'Arts of the Americas',
 'Arts of the Americas',
 'Arts of the Americas',
 'Arts of the Americas',
 'Arts of the Americas',
 'Arts of the Americas',
 'Arts of Asia',
 'Arts of Asia',
 'Arts of Asia',
 'Arts of Asia',
 'Arts of Asia']

In [39]:
# I combine the artwork data from both the CSV and the API into a single DataFrame, 
# ensuring that all relevant information is consolidated for my code.

api_art_data = pd.DataFrame(all_art_data['data'])

csv_art_data = pd.read_csv("AIC_data.csv")

combined_art_data = pd.concat([csv_art_data, api_art_data], ignore_index=True)

In [40]:
# I list all unique department titles in the combined dataset to get an overview of 
# the available art categories."

all_departments = combined_art_data["department_title"].unique()
print("All Departments:", all_departments)

all_classifications = combined_art_data["classification_titles"].explode().unique()
print("\nAll Classifications:", all_classifications)

All Departments: ['Painting and Sculpture of Europe' 'Arts of the Americas' 'Modern Art'
 'Contemporary Art' 'Arts of Asia' 'Photography and Media'
 'Prints and Drawings' 'Applied Arts of Europe' 'Textiles'
 'Arts of Africa' 'Architecture and Design' nan
 'Arts of the Ancient Mediterranean and Byzantium']

All Classifications: [nan 'drinking vessel' 'cocktail shaker' 'tray' 'american decorative arts'
 'decorative arts' 'painting' 'american arts' 'container' 'silver'
 'furniture accessory' 'sculpture' 'bronze' 'metal' 'asian art'
 'bust/head' 'stuccowork' 'sandstone' 'stone' 'reliquary' 'schist']


In [18]:
# The combined dataset is filtered to include only the selected departments relevant to 
# my exhibition, ensuring that only interesting artworks are considered.

selected_departments = ["Modern Art", "Contemporary Art", "Photography and Media", "Prints and Drawings", "Textiles"]

department_filter = combined_art_data["department_title"].isin(selected_departments)

filtered_art_data = combined_art_data[department_filter]

In [ ]:
filtered_art_data

In [ ]:
# Calculating the total number of artworks available in the filtered dataset.

artwork_count = len(filtered_art_data)
artwork_count

In [ ]:
# Counting the number of artworks in each selected department, 
# which provides insight into the distribution of artworks across categories.

department_counts = filtered_art_data["department_title"].value_counts()

department_counts

In [ ]:
# A list of artwork titles is created from the filtered dataset, and I print them out to quickly see the available artworks.

titles = filtered_art_data["title"].tolist()
for title in titles:
    print(title)

In [21]:
# Art-to-Music Mapping Concept

# Group 1: Abstract and Experimental
# Departments: Modern Art, Contemporary Art
# Genres: ambient, experimental, trip-hop, electronic, idm
# Vibe: Abstract, innovative, and boundary-pushing, these genres provide a textured and atmospheric soundscape, perfect for the 
# unconventional nature of modern and contemporary works. The music offers an abstract exploration that reflects the introspective and 
# avant-garde qualities of this art.

# Group 2: Textiles and Cultural Arts
# Departments: Textiles, Arts of the Americas, Arts of Africa
# Genres: world-music, new-age, ethereal, folk, americana
# Vibe: Rooted in cultural heritage and natural themes, these genres evoke an expansive, meditative quality. They emphasize tradition and 
# a connection to nature, while still feeling fresh, making them a fitting choice for textile art and culturally rich works from the
# Americas and Africa.

# Group 3: Historical and Classical Arts
# Departments: Painting and Sculpture of Europe, Applied Arts of Europe, Arts of the Ancient Mediterranean and Byzantium
# Genres: classical, baroque, medieval, opera, blues
# Vibe: Timeless and sophisticated, these genres embody the rich history and craftsmanship of European and ancient Mediterranean art. 
# The music is regal and intricate, reflecting the elegance and historical depth of these classical and applied art forms.

# Group 4: Photography and Media
# Departments: Photography and Media, Prints and Drawings
# Genres: cinematic, dark-ambient, neo-classical, lo-fi, soundscapes
# Vibe: Emotive and introspective, these genres are ideal for storytelling and capturing mood. The music enhances the narrative and 
# contemplative qualities of photography and intricate printmaking, creating an immersive experience that highlights the evocative 
# and detailed nature of these media.

# Group 5: Asian Art and Fusion
# Departments: Arts of Asia
# Genres: traditional, k-pop, j-pop, electronic
# Vibe: A blend of tradition and modernity, these genres capture the cultural richness and contemporary influence in Asian art. 
# This fusion of traditional and modern sounds is both dynamic and respectful of heritage, creating a soundtrack that complements 
# Asian art’s unique blend of history and innovation.

In [23]:
# Mapping of art styles to music genres

art_to_music_map = {
    "Modern Art": ["ambient", "experimental", "trip-hop", "electronic", "idm"],
    "Contemporary Art": ["downtempo", "synth-pop", "chill", "minimal-techno", "new-age"],
    "Photography and Media": ["dark-ambient", "electronic", "lo-fi", "cinematic", "ambient"],
    "Prints and Drawings": ["ethereal", "ambient", "downtempo", "soundscapes", "neo-classical"],
    "Textiles": ["new-age", "ethereal", "drone", "ambient"],
    "Arts of the Americas": ["folk", "americana"],
    "Arts of Africa": ["afrobeat", "african"],
    "Painting and Sculpture of Europe": ["classical", "baroque"],
    "Applied Arts of Europe": ["jazz", "blues"],
    "Arts of the Ancient Mediterranean and Byzantium": ["medieval", "classical"],
    "Arts of Asia": ["traditional", "k-pop", "j-pop", "electronic"]
}


In [24]:
# Loop which generates and prints music recommendations for each art category based on 
# the genre mapping, demonstrating whcich music will compliment the art experience.

for art_category, genre_seeds in art_to_music_map.items():
    print(f"\nMusic Recommendations for {art_category}:")

    recommendations = sp.recommendations(seed_genres=genre_seeds, limit=5)

    for track in recommendations['tracks']:
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        print(f" ♫ {track_name} by {artist_name}")

HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 5, 'seed_genres': 'ambient,experimental,trip-hop,electronic,idm'} returned 401 due to The access token expired



Music Recommendations for Modern Art:


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/recommendations?limit=5&seed_genres=ambient%2Cexperimental%2Ctrip-hop%2Celectronic%2Cidm:
 The access token expired, reason: None

In [ ]:
# At this point, I wanted to bring in images of the art pieces in my chosen categories so I can start creating randomised mixtures of the art
# and music. I know from the AIC API that they provide URLs for images and jupyter is able to show images. However, I realised at this point
# that in the CSV data set, there is no column for Image URLs. I tired getting all the image ids from my art data set and see if there is a 
# way for me to get URLs using those image ids. Unfortunately, that become too confusing and convoluted for me to understand so I pivioted 
# to collecting images straight from the API, still using my parameters. 

In [ ]:
# I defined the create_virtual_exhibition function, which takes filtered artwork data 
# and selected categories to display artworks and generate music recommendations.

# Inside this function, I retrieve artworks based on selected categories, display them, 
# fetch corresponding music recommendations, and create a Spotify playlist with the recommended tracks.
    

def create_virtual_exhibition(filtered_art_data, selected_categories, num_images=3, num_songs=5):
    track_uris = []
    print(f"Virtual Exhibition of {selected_categories}\n")

    for category in selected_categories:
        print(f"\nExhibition Category: {category}")

        # Getting artworks in the chosen category
        artworks_in_category = filtered_art_data[filtered_art_data["department_title"] == category]

        if artworks_in_category.empty:
            print(f"No artworks found for category: {category}")
            continue
        
        # Randomly select images from the category
        sample_artworks = artworks_in_category.sample(n=min(num_images, len(artworks_in_category)))
        for _, artwork in sample_artworks.iterrows():
            print(f"Title: {artwork['title']} - Artist: {artwork.get('artist_display', 'Unknown')}")
            image_url = artwork.get("image_url")
            if pd.notna(image_url) and image_url:
                display(Image(url=image_url, width=300))

        # Getting music recommendations based on the genre seeds for each category
        genre_seeds = art_to_music_map.get(category, [])
        recommendations = sp.recommendations(seed_genres=genre_seeds, limit=num_songs)
        for track in recommendations['tracks']:
            track_name = track['name']
            artist_name = track['artists'][0]['name']
            print(f" - {track_name} by {artist_name}")
            track_uris.append(track['uri'])

    # Create a Spotify playlist with the collected track URIs
    if track_uris:  # Check if there are any track URIs
        playlist = sp.user_playlist_create(user=username, name="Virtual Gallery Playlist", public=True)
        sp.playlist_add_items(playlist_id=playlist['id'], items=track_uris)
        print("\nSpotify Playlist Link:", playlist['external_urls']['spotify'])
    else:
        print("No songs to add to the playlist.")

In [ ]:
#"The exhibition function is called with specific categories, demonstrating how the 
# virtual exhibition works by displaying artworks and generating a music playlist that 
# aligns with those themes.

selected_categories = ["Modern Art", "Contemporary Art"]
create_virtual_exhibition(filtered_art_data, selected_categories, num_images=5, num_songs=5)

In [ ]:
selected_categories = ["Photography and Media", "Textiles"]
create_virtual_exhibition(filtered_art_data, selected_categories, num_images=5, num_songs=5)

In [ ]:
# In summary, this project successfully combines art and music to create an engaging 
# virtual exhibition experience. By structuring the code to load, filter, and display 
# artworks while generating tailored music recommendations, I aim to enhance the overall 
# enjoyment of the exhibition.